# 02 — Optimization with PyTorch (4×4 Sudoku)

This notebook builds on **Chapter 01** (tensor basics + differentiable constraints).  
Here we focus on **how** an optimizer turns those constraints into an actual Sudoku solution.

> Key idea: we do **not** train a neural network.  
> We directly optimize a **tensor of logits** whose softmax becomes the probability tensor **P**.

The 4×4 case primarily serves as a didactic example to illustrate
the continuous relaxation and optimization dynamics.
Due to the small problem size, convergence is typically very fast,
even for puzzles with many unknown cells.


---
## 2.1 Recap: The Constraint Landscape
A valid Sudoku solution is defined by constraints:
- **Row uniqueness**: each digit appears exactly once per row
- **Column uniqueness**: each digit appears exactly once per column
- **Block uniqueness**: each digit appears exactly once per block
- **Givens**: given cells must keep the correct digit
- Optional: **entropy** to encourage confident (nearly one-hot) predictions


In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

torch.set_printoptions(precision=4, sci_mode=False)
device = torch.device("cpu")


---
## 2.2 What Does It Mean to Optimize **P**?
We represent a 4×4 Sudoku as a tensor of probabilities:

- **P** has shape **(4, 4, 4)**
- `P[r, c, k]` is the probability that cell `(r, c)` contains digit `k+1`.

During optimization, we do **not** update `P` directly.  
Instead we optimize **logits** `Z` and compute:

\[ P = \text{softmax}(Z / T) \]

where `T` is a temperature (optionally annealed over time).


---
## 2.3 Parameterization: Logits → Softmax → Probabilities

Why optimize logits?
- logits are unconstrained real numbers
- softmax turns them into a valid probability distribution per cell
- probabilities stay well-behaved during gradient updates


## Help Functions
- `probs_from_logits` applies a softmax over the digit dimension `k`,
  ensuring that the probabilities for each cell sum to 1.
- `argmax` selects the most likely digit per cell and is used only for
  visualization, not during optimization. argmax returns the first occurrence of the maximum value.
- `loss-dict` calls the loss function `sudoku_losses`and prints the calclated losses for P 

In [ ]:
def probs_from_logits(Z, T=1.0):
    # Z: (4,4,4) logits
    # returns P: (4,4,4) probs
    return F.softmax(Z / T, dim=2)

def pretty_grid_from_probs(P):
    # takes argmax per cell, returns digits 1..4
    return P.argmax(dim=2) + 1

def loss_dict(P, givens_mask, givens_target, weights):
    w_row, w_col, w_blk, w_giv, w_ent = weights
    L_row, L_col, L_blk, L_giv, L_ent = sudoku_losses(P, givens_mask, givens_target)
    L_total = w_row*L_row + w_col*L_col + w_blk*L_blk + w_giv*L_giv + w_ent*L_ent
    return {
        "L_row": L_row,
        "L_col": L_col,
        "L_blk": L_blk,
        "L_giv": L_giv,
        "L_ent": L_ent,
        "L_total": L_total,
    }

def print_losses(tag, d):
    # .item() macht aus tensor(...) einen float, ohne grad_fn-Anzeige
    print(
    tag,
    {k: f"{v.detach().cpu().item():.3f}" for k, v in d.items()},
    "\n"
)




---
## 2.4 The Optimizer (Adam) — High-Level View

Each iteration does:
1. logits `Z` → probabilities `P`
2. compute constraint losses
3. sum into `L_total`
4. backpropagate gradients w.r.t. `Z`
5. `optimizer.step()` updates `Z`

Next, we implement the constraint losses.


In [ ]:
def sudoku_losses(P, givens_mask, givens_target, eps=1e-8):
    """
    P: (4,4,4) probabilities over digits 1..4 (k dimension last)
    givens_mask: (4,4) bool, True where a clue is given
    givens_target: (4,4) long, values in [0..3] for digit indices (digit-1)
    """
    # Row uniqueness: for each row i and digit k, sum over columns j should be 1
    row_sum = P.sum(dim=1)                 # (4,4)
    L_row = ((row_sum - 1.0) ** 2).sum()

    # Column uniqueness: for each col j and digit k, sum over rows i should be 1
    col_sum = P.sum(dim=0)                 # (4,4)
    L_col = ((col_sum - 1.0) ** 2).sum()

    # Block uniqueness: 2×2 blocks
    L_blk = 0.0
    for br in (0, 2):
        for bc in (0, 2):
            blk = P[br:br+2, bc:bc+2, :]   # (2,2,4)
            blk_sum = blk.sum(dim=(0,1))   # (4,)
            L_blk = L_blk + ((blk_sum - 1.0) ** 2).sum()

    # Givens loss: encourage given cells to keep probability 1 at the given digit
    # pick probabilities at given positions
    given_probs = P[givens_mask]                       # (num_givens, 4)
    given_targets = givens_target[givens_mask]         # (num_givens,)
    # negative log-likelihood (cross entropy) on givens
    L_giv = F.nll_loss((given_probs + eps).log(), given_targets, reduction='sum')

    # Entropy: encourage low entropy (more confident predictions)
    ent = -(P * (P + eps).log()).sum(dim=2)            # (4,4)
    L_ent = ent.sum()

    return L_row, L_col, L_blk, L_giv, L_ent


---
## 2.5 One Optimization Step (Concrete!)
We demonstrate a single optimizer step on an **almost solved** 4×4 Sudoku.

Sudoku Puzzle:
```
[1, 2, 3, 4]
[3, 4, 1, 2]
[2, 1, 4, 3]
[4, 3, 0, 1]
```

Only one cell [3,2| is unknown. This makes it easy to see how the optimizer shifts
probabilities away from inconsistent digits and towards the only consistent digit.

If we translate the Sudoku Puzzle to `P`, we get the following cube:

<img src="images/02_P_Cube_4x4x4.jpg" alt="4×4×4 Sudoku as a probability tensor cube" width="200"/>

- Gray cubes represent a probability of `0.25` (uniform uncertainty)
- Beige cubes represent `0.0`
- Black cubes represent given digits with value `1.0`



In [ ]:
# Almost solved 4×4 Sudoku (0 = unknown)
# Digits are 1..4
grid = torch.tensor([
    [1, 2, 3, 4],
    [3, 4, 1, 2],
    [2, 1, 4, 3],
    [4, 3, 0, 1],
], dtype=torch.long)

# Build givens mask + targets (targets are digit-1 in [0..3])
givens_mask = grid != 0
givens_target = (grid-1).clamp(min=0)

grid, givens_mask, givens_target


In [ ]:
# Initialize logits Z.
# For given cells, we initialize logits such that the correct digit is strongly preferred.
# For the unknown cell, logits start at 0 -> uniform probabilities after softmax.

Z = torch.zeros((4,4,4), dtype=torch.float32, device=device)

# Strongly bias given cells (large positive logit for the correct digit, negative for others)
high = 6.0
low = -6.0
for r in range(4):
    for c in range(4):
        if givens_mask[r,c]:
            k = int(givens_target[r,c].item())  # 0..3
            Z[r,c,:] = low
            Z[r,c,k] = high

# Check probabilities at the unknown cell (3,2)
P0 = probs_from_logits(Z, T=1.0)
Z, P0, pretty_grid_from_probs(P0)


In [ ]:
# Compute losses BEFORE the step
L_row, L_col, L_blk, L_giv, L_ent = sudoku_losses(P0, givens_mask, givens_target)
L_row, L_col, L_blk, L_giv, L_ent


In [ ]:
# One optimizer step
weights = (1.0, 1.0, 1.2, 2.0, 0.01)

Z_step = Z.clone().detach().requires_grad_(True)
opt = torch.optim.Adam([Z_step], lr=0.3)

# BEFORE step
P = probs_from_logits(Z_step, T=1.0)
print ("P-before", P,"\n", pretty_grid_from_probs(P), "\n")
d0 = loss_dict(P, givens_mask, givens_target, weights)
print_losses("P  (before)", d0)

# update
opt.zero_grad()
d0["L_total"].backward()
opt.step()

# AFTER step
P1 = probs_from_logits(Z_step, T=1.0)  # detach nicht nötig fürs Anzeigen
d1 = loss_dict(P1, givens_mask, givens_target, weights)
print ("P1-after", P1,"\n", pretty_grid_from_probs(P1), "\n")
print_losses("P1 (after) ", d1)

# Deltas
print("Δ (after - before):",
      {k: float((d1[k]-d0[k]).detach().cpu().item()) for k in d0.keys()})


### 2.6 For Loop
Let's do a for Loop to show the convergencey more cleraly. 
You will oberserve that cell P(3,2) converges quickly from 

`[0.25,0.25,0.25]`
to
`[0.0080, 0.9761, 0.0080, 0.0080]`



In [ ]:
weights = (1.0, 1.0, 1.2, 2.0, 0.01)

Z_step = Z.clone().detach().requires_grad_(True)
opt = torch.optim.Adam([Z_step], lr=0.3)

unknown_rc = (3, 2)

# Baseline (Iteration 0)
P0 = probs_from_logits(Z_step, T=1.0)
d_base = loss_dict(P0, givens_mask, givens_target, weights)

print("Iter 0")
print("Grid:\n", pretty_grid_from_probs(P0))
print("Unknown cell probs:", P0[unknown_rc[0], unknown_rc[1], :].detach().cpu())
print_losses("Losses", d_base)
print()

# Loop
steps = 10
print_every = 1  # z.B. 1 = jede Iteration, 5 = jede 5te

for it in range(1, steps + 1):
    P = probs_from_logits(Z_step, T=1.0)
    d = loss_dict(P, givens_mask, givens_target, weights)

    opt.zero_grad()
    d["L_total"].backward()
    opt.step()

    # Anzeige nach dem Update
    P_after = probs_from_logits(Z_step, T=1.0)
    d_after = loss_dict(P_after, givens_mask, givens_target, weights)

    if it % print_every == 0:
        print(f"Iter {it}")
        print("Grid:\n", pretty_grid_from_probs(P_after))
        print("Unknown cell probs:", P_after[unknown_rc[0], unknown_rc[1], :].detach().cpu())

        # hübsch: before/after/delta zur Baseline
        deltas = {k: float((d_after[k] - d_base[k]).detach().cpu().item()) for k in d_base.keys()}
        print_losses("Losses", d_after)
        print("Δ vs Iter0:", {k: f"{v:+.3f}" for k, v in deltas.items()})
        print()


**Interpretation**  
The optimizer does not “know” Sudoku. It follows gradients created by the constraints.  
Digits that violate row/column/block consistency are pushed down, while the consistent digit is pushed up.

In a less constrained puzzle, this process takes many iterations and benefits from entropy + temperature control.


---
## 2.6 Entropy and Temperature Annealing

- **Entropy loss** encourages probabilities to become confident (close to one-hot).
- **Temperature** controls how sharp softmax outputs are:
  - high `T` → softer probabilities (more exploration)
  - low `T` → sharper probabilities (more exploitation)

A common strategy is to start with a higher temperature and slowly decrease it.


In [ ]:
def optimize_sudoku(Z_init, givens_mask, givens_target, steps=300, lr=0.2,
                   T_start=1.5, T_end=0.6, w_row=1.0, w_col=1.0, w_blk=1.2, w_giv=2.0, w_ent=0.01,
                   save_P_every=5):
    Z = Z_init.clone().detach().requires_grad_(True)
    opt = torch.optim.Adam([Z], lr=lr)

    hist = {
        "L_total": [], "L_row": [], "L_col": [], "L_blk": [], "L_giv": [], "L_ent": [], "T": [], 
        "P_snapshots": {}   # step -> P
    }

    for t in range(steps):
        # linear temperature schedule
        alpha = t / max(1, steps-1)
        T = (1-alpha)*T_start + alpha*T_end

        P = probs_from_logits(Z, T=T)
        L_row, L_col, L_blk, L_giv, L_ent = sudoku_losses(P, givens_mask, givens_target)
        L_total = w_row*L_row + w_col*L_col + w_blk*L_blk + w_giv*L_giv + w_ent*L_ent

        opt.zero_grad()
        L_total.backward()
        opt.step()

        hist["L_total"].append(float(L_total.detach().cpu()))
        hist["L_row"].append(float(L_row.detach().cpu()))
        hist["L_col"].append(float(L_col.detach().cpu()))
        hist["L_blk"].append(float(L_blk.detach().cpu()))
        hist["L_giv"].append(float(L_giv.detach().cpu()))
        hist["L_ent"].append(float(L_ent.detach().cpu()))
        hist["T"].append(float(T))
        # store intermediate P
        if t % save_P_every == 0 or t == steps - 1:
            hist["P_snapshots"][t] = P.detach().cpu()

    return Z.detach(), hist


In [ ]:
# Run the full optimization on the same "almost solved" puzzle (converges quickly)
Z_final, hist = optimize_sudoku(Z, givens_mask, givens_target, steps=10, lr=0.3, T_start=1.2, T_end=0.7)
P_final = probs_from_logits(Z_final, T=0.7)
pretty_grid_from_probs(P_final)

---
## 2.7 Convergence Behavior and Failure Modes

Typical issues when scaling to harder puzzles:
- **Local minima**: the optimizer gets stuck in a partially consistent state
- **Learning rate too high**: oscillations or instability
- **Weights poorly tuned**: one constraint dominates and blocks progress
- **Entropy too strong too early**: forces premature, wrong decisions

Below we visualize convergence (loss over iterations) for this simple example.


In [ ]:
plt.figure()
plt.plot(hist["L_total"])
plt.title("Total loss over iterations")
plt.xlabel("iteration")
plt.ylabel("L_total")
plt.show()

plt.figure()
plt.plot(hist["L_ent"])
plt.title("Entropy loss over iterations")
plt.xlabel("iteration")
plt.ylabel("L_ent")
plt.show()

plt.figure()
plt.plot(hist["T"])
plt.title("Temperature schedule")
plt.xlabel("iteration")
plt.ylabel("T")
plt.show()


### 2.7.1 Observations 
After 4 steps we have already a Total_Loss of almost 0 which shows that the solution is found very quickly. 

In this context, a solution is considered found once the `argmax`
projection yields a grid that satisfies all Sudoku constraints.


Let's try with more unknowns and analyze the results.


### 2.7.2 More Unkowns
In this example we will select more unknowns in order to show how the optimizer works for a mutlidimensional field.
We select as Puzzle following grid
```
[0, 0, 3, 1]
[1, 0, 0, 0]
[0, 2, 1, 0]
[0, 0, 4, 0]
```
So our unknowns are P(0,0), P(0,1), P(1,1), P(1,2), P(1,3), P(2,0), P(2,3), P(3,0), P(3,1) and P(3,3)

In [ ]:
# Almost solved 4×4 Sudoku (0 = unknown)
# Digits are 1..4
grid = torch.tensor([
    [0, 0, 3, 1],
    [1, 0, 0, 0],
    [0, 2, 1, 3],
    [0, 0, 4, 0],
], dtype=torch.long)

# Build givens mask + targets (targets are digit-1 in [0..3])
givens_mask = grid != 0
givens_target = (grid-1).clamp(min=0)

grid, givens_mask, givens_target


In [ ]:
# Initialize logits Z.
# For given cells, we initialize logits such that the correct digit is strongly preferred.
# For the unknown cell, logits start at 0 -> uniform probabilities after softmax.

Z = torch.zeros((4,4,4), dtype=torch.float32, device=device)

# Strongly bias given cells (large positive logit for the correct digit, negative for others)
high = 6.0
low = -6.0
for r in range(4):
    for c in range(4):
        if givens_mask[r,c]:
            k = int(givens_target[r,c].item())  # 0..3
            Z[r,c,:] = low
            Z[r,c,k] = high

# Check probabilities at the unknown cell (3,2)
P0 = probs_from_logits(Z, T=1.0)
Z, P0, pretty_grid_from_probs(P0)

In [ ]:
# Run the full optimization on the same "almost solved" puzzle (converges quickly)
Z_final, hist = optimize_sudoku(Z, givens_mask, givens_target, steps=10, lr=0.3, T_start=1.2, T_end=0.7)
P_final = probs_from_logits(Z_final, T=0.7)
print("Final solution:\n",pretty_grid_from_probs(P_final))


In [ ]:
plt.figure()
plt.plot(hist["L_total"])
plt.title("Total loss over iterations")
plt.xlabel("iteration")
plt.ylabel("L_total")
plt.show()

plt.figure()
plt.plot(hist["L_ent"])
plt.title("Entropy loss over iterations")
plt.xlabel("iteration")
plt.ylabel("L_ent")
plt.show()

plt.figure()
plt.plot(hist["T"])
plt.title("Temperature schedule")
plt.xlabel("iteration")
plt.ylabel("T")
plt.show()

### 2.7.3 Observations (More Unknowns)

With more unknown cells, the optimization starts from a much larger feasible space.
As a result, the total loss typically decreases **less steeply** in the first iterations
compared to the “almost solved” case.

However, for 4×4 Sudoku the constraint landscape is still very small and strongly structured.
In this example, the optimizer reaches a discrete-looking solution quickly:
after about 10 iterations, the `argmax` projection already yields a valid Sudoku grid.

Note that we optimize a **continuous relaxation** `P`, while the displayed grid is obtained via `argmax`.
Therefore, “finding the solution” here means: *the `argmax` projection becomes consistent with all constraints*,
even though the underlying probabilities may still continue to sharpen slightly afterwards.

In practice, 4×4 Sudoku often converges very fast under this relaxation,
because the problem is small (only 16 cells) and the constraints (rows/cols/blocks) are very strong.
More interesting convergence behavior typically appears when scaling to 9×9.



### 2.7.4 Limitations of the 4×4 Case

- The optimization landscape is very small and strongly constrained.
- Differences in puzzle difficulty are compressed into very few iterations.
- Loss curves alone are not sufficient to characterize convergence behavior.

These limitations motivate the extension to 9×9 Sudoku.

In the next chapter, we extend the same formulation to 9×9 Sudoku.
At this scale, the optimization dynamics become significantly more interesting,
with longer plateaus, stronger symmetry effects, and slower convergence.



---
## Next
In Chapter 03 we scale the same ideas to **9×9 Sudoku**:
- tensor becomes `(9, 9, 9)`
- blocks become `3×3`
- optimization becomes harder and needs careful tuning
